In [1]:
import numpy as np
import pandas as pd

import cv2
from sklearn.model_selection import train_test_split
from sklearn import metrics # to evaluate classification accuracy
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import matplotlib.image as img
import seaborn as sns
import random

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras import initializers
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array
from tensorflow.keras.models import Model, load_model,Sequential
from tensorflow.keras.layers import (Input, Dense, Dropout, Activation,
                                     Flatten, BatchNormalization, Conv2D,
                                     MaxPooling2D,GlobalAveragePooling2D)
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.preprocessing import LabelBinarizer

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
from tqdm import tqdm

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras import initializers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model,Sequential
from tensorflow.keras.layers import (Input, Dense, Dropout, Activation,
                                     Flatten, BatchNormalization, Conv2D,
                                     MaxPooling2D,GlobalAveragePooling2D)
import tensorflow as tf
from tensorflow.keras import datasets, models, layers, utils, activations, losses, optimizers, metrics
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from glob import glob
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import pandas as pd

#從路徑讀取圖像資料, 圖像資訊儲存成d_x,類別資訊儲存成d_y
DATASET_PATH = "./jpg/"
# DATASET_PATH = r"D:\py\mid\cnn\training_set\training_set" +'//'

d_x = []
d_y = []

df = pd.read_csv('./data2.csv',dtype={'category':str})
# df = df.drop(['Context'], axis='columns')
df= df.drop(columns=['Times'])
df= df.drop(columns=['Impressions'])
df= df.drop(columns=['ExposureCTR'])
# df= df.drop(columns=['File_name'])
df.columns

nowpath = os.getcwd()
try:   
    shutil.rmtree('././.ipynb_checkpoints')
#     print('clear   ././.ipynb_checkpoints')
except:
    pass
df['category']=df['category'].astype(int)
# for category in sorted(os.listdir(DATASET_PATH)):
#     print('loading category: '+str(category))
for fname in os.listdir(DATASET_PATH):
    try:
#             img = cv2.imread(DATASET_PATH, 1)
        img = cv2.imread(DATASET_PATH+fname, 1)
        img = cv2.resize(img, (224,224))
        d_x.append(np.reshape(img, [224,224,3]))
        d_y.append(str(fname))
    except:
            pass
print(len(d_x),len(d_y))

144 144


In [2]:
# import os 
# os.listdir(DATASET_PATH+fname)
# d_x

In [3]:
d_x = np.array(d_x).astype('float32') # 避免有些圖像像素太大, scale to 0-1 

In [4]:
df

,File_name,category
0,lB_5YsAV77k.jpg,1
1,5rnLJjhAn78.jpg,1
2,0qJSTqfVEvE.jpg,1
3,YW2RNQlMddw.jpg,1
4,iYxgUD48QBg.jpg,1
...,...,...
139,NCsOGXpAOs8.jpg,0
140,pfrUm2WB49s.jpg,0
141,6omkNL68UEg.jpg,0
142,WbKoxonx9T4.jpg,0


In [5]:
d_y2=pd.DataFrame(d_y,columns=['File_name'])
d_y3=pd.merge(d_y2,df,on='File_name')
d_y3
d_y=d_y3.copy()
d_y=d_y.drop(columns='File_name')
d_y

,category
0,1
1,1
2,0
3,1
4,0
...,...
139,1
140,0
141,1
142,1


#### Get data

In [6]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
d_y=pd.DataFrame(d_y)
d_y= labelencoder.fit_transform(d_y)
d_y

/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1,
       0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1,
       1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0,
       1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1])

In [7]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
d_y=pd.DataFrame(d_y)
d_y= labelencoder.fit_transform(d_y)
d_y
len(d_y)
# y_data_list = pd.DataFrame(d_y, columns=['label'])
# y_data = y_data_list['label']
# d_y_onehot = tf.keras.utils.to_categorical(y_data, num_classes=2, dtype='float32')
# print(d_y_onehot)

144

In [8]:
# train = df[:int(len(df)*0.8)]
# test = df[int(len(df)*0.8):]
# train.shape, test.shape

# 

train_x = d_x[:int(len(df)*0.8)]
train_y = d_y[:int(len(df)*0.8)]
vald_x = d_x[int(len(df)*0.8):]
vald_y = d_y[int(len(df)*0.8):]

# X_train.shape, y_train.shape

In [9]:
# train_x = pd.get_dummies(train_x)
# train_y = pd.get_dummies(train_y)
# vald_x = pd.get_dummies(vald_x)
# vald_y = pd.get_dummies(vald_y)

In [10]:
img_rows, img_cols = train_x.shape[1],train_x.shape[2] 
input_shape = (img_rows, img_cols, 3) 

In [11]:
IMG_SIZE = 224
tf.keras.backend.clear_session()
base_model=tf.keras.applications.EfficientNetB7(include_top=False, # include classifiser ?
                                                  weights='imagenet', 
                                                  input_shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model.output

x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.2)(x)
predictions = layers.Dense(1, activation='sigmoid')(x)


model = models.Model(base_model.input, predictions)


optimizer = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
model.compile(optimizer = optimizer , 
              loss = "binary_crossentropy", 
              metrics=["accuracy"])



In [12]:
datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=10, 
    width_shift_range=0.1,
    height_shift_range=0.1,  
    horizontal_flip=True,  
    vertical_flip=False,)  

from tensorflow.keras import callbacks

# 定義 learning rate 根據 epoch 要如何變動
def schedule(epoch):  
    if epoch < 10:
        return 0.0005
    elif epoch < 30:
        return 0.0003
    elif epoch < 50:
        return 0.0001
    else:
        return 0.00005
# 建立 LearningRateScheduler

lr_schedule = callbacks.LearningRateScheduler(schedule, verbose=1)
model_mckp = tf.keras.callbacks.ModelCheckpoint('Model/', # 模型輸出路徑
                                             monitor='val_accuracy',
                                             save_best_only=True,
                                             verbose=1,)

In [15]:
#datagen.fit(train_x)

history = model.fit(datagen.flow(train_x,train_y, 
                                 batch_size=8),
                              steps_per_epoch=len(train_x) / 8, 
                              epochs=50,
                              validation_data=(vald_x,vald_y),
                              callbacks=[model_mckp,lr_schedule])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.0005.
Epoch 1/50
15/14 [===============================] - ETA: 0s - loss: 0.7821 - accuracy: 0.4609
Epoch 00001: val_accuracy improved from -inf to 0.51724, saving model to Model/
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: Model/assets
15/14 [===============================] - 96s 6s/step - loss: 0.7821 - accuracy: 0.4609 - val_loss: 0.7423 - val_accuracy: 0.5172

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0005.
Epoch 2/50
15/14 [===============================] - ETA: 0s - loss: 0.6368 - accuracy: 0.6870
Epoch 00002: val_accuracy did not improve from 0.51724
15/14 [===============================] - 5s 325ms/step - loss: 0.6368 - accuracy: 0.6870 - val_loss: 1.0688 - val_accuracy

In [ ]:
d_x

In [ ]:
d_x.shape

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
(115,224,244,3)